<a href="https://colab.research.google.com/github/emilymacris/ds3001group/blob/main/michiganpredict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone https://github.com/emilymacris/ds3001group.git

Cloning into 'ds3001group'...
remote: Enumerating objects: 616, done.
remote: Counting objects: 100% (262/262), done.
remote: Compressing objects: 100% (176/176), done.
remote: Total 616 (delta 153), reused 147 (delta 83), pack-reused 354 (from 1)
Receiving objects: 100% (616/616), 52.90 MiB | 16.54 MiB/s, done.
Resolving deltas: 100% (303/303), done.


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Read in files

outcomes = pd.read_excel("./ds3001group/dataverse_files/WinningPartyByCounties.xlsx") # party outcome at county level

rawdemo2012 = pd.read_excel("./ds3001group/dataverse_files/demographicsallstates2012.xlsx") # demographic variables at county level 2012
rawdemo2016 = pd.read_excel("./ds3001group/dataverse_files/demographicsallstates2016.xlsx") # demographic variables at county level 2016
rawdemo2020 = pd.read_excel("./ds3001group/dataverse_files/demographicsallstates2020.xlsx") # demographic variables at county level 2020

mipoverty = pd.read_csv("./ds3001group/michigan/mipoverty.csv") # poverty variable at county level -- Michigan
mipoverty = mipoverty.drop([0,1, 38, 39]) # Drop national and state level statistics

prediction = pd.read_excel("./ds3001group/2022datafinal.xlsx")
prediction.head(10)

,GIS Join Match Code,Data File Year,State/US-Abbreviation (USPS),Region Code,Division Code,State Name,State Code,County Name,County Code,County Subdivision Code,...,Aggregate income in the past 12 months (in 2022 inflation-adjusted dollars),Total.5,In labor force,In labor force Percentage,In labor force: Civilian labor force,In labor force: Civilian labor force: Employed,In labor force: Civilian labor force: Unemployed,In labor force: Armed Forces,Not in labor force,Not in labor force Percentage
0,G0400010,2018-2022,AZ,NaN,NaN,Arizona,4,Apache County,1,NaN,...,2076142800,47146,28106,0.596148,27163,26411,752,943,19040,0.403852
1,G0400030,2018-2022,AZ,NaN,NaN,Arizona,4,Cochise County,3,NaN,...,9081671900,189569,111766,0.589580,111079,107254,3825,687,77803,0.410420
2,G0400050,2018-2022,AZ,NaN,NaN,Arizona,4,Coconino County,5,NaN,...,581565400,20399,8968,0.439629,8968,8452,516,0,11431,0.560371
3,G0400070,2018-2022,AZ,NaN,NaN,Arizona,4,Gila County,7,NaN,...,555512400,18046,9028,0.500277,9028,8242,786,0,9018,0.499723
4,G0400090,2018-2022,AZ,NaN,NaN,Arizona,4,Graham County,9,NaN,...,1716337800,47261,26459,0.559849,26434,24856,1578,25,20802,0.440151
5,G0400110,2018-2022,AZ,NaN,NaN,Arizona,4,Greenlee County,11,NaN,...,228401300,8376,3888,0.464183,3888,3744,144,0,4488,0.535817
6,G0400120,2018-2022,AZ,NaN,NaN,Arizona,4,La Paz County,12,NaN,...,499854700,15203,8047,0.529303,8043,7495,548,4,7156,0.470697
7,G0400130,2018-2022,AZ,NaN,NaN,Arizona,4,Maricopa County,13,NaN,...,3278894900,94627,53521,0.565600,53147,49455,3692,374,41106,0.434400
8,G0400150,2018-2022,AZ,NaN,NaN,Arizona,4,Mohave County,15,NaN,...,934715300,28036,15691,0.559673,15631,15072,559,60,12345,0.440327
9,G0400170,2018-2022,AZ,NaN,NaN,Arizona,4,Navajo County,17,NaN,...,712518400,20956,10713,0.511214,10713,10288,425,0,10243,0.488786


In [ ]:
# Filter files for Michigan

mioutcomes = outcomes[outcomes['state'] == 'MICHIGAN'] # filtered outcome variable for Michigan

demo2012 = rawdemo2012[rawdemo2012['State Name'] == 'Michigan'] # filtered demographic variables for Michigan
demo2016 = rawdemo2016[rawdemo2016['State Name'] == 'Michigan']
demo2020 = rawdemo2020[rawdemo2020['State Name'] == 'Michigan'] # filtered demographic variables for Michigan

In [ ]:
mioutcomes2012 = mioutcomes[mioutcomes['year'] == 2012]
mioutcomes2016 = mioutcomes[mioutcomes['year'] == 2016]
mioutcomes2020 = mioutcomes[mioutcomes['year'] == 2020] # separated outcomes variable by year

mipoverty2012 = mipoverty[mipoverty['Year'] == 2012]
mipoverty2016 = mipoverty[mipoverty['Year'] == 2016]
mipoverty2020 = mipoverty[mipoverty['Year'] == 2020] # separated poverty variable by year

In [ ]:
# Merge

election2012 = pd.merge(demo2012, mipoverty2012, left_on='County Name', right_on='Name', how='outer') # Merge poverty with demographics
election2012 = pd.merge(election2012, mioutcomes2012, left_on='ID', right_on='county_fips', how='outer') # Merge outcomes with election2012
# now have a complete file for 2012

election2016 = pd.merge(demo2016, mipoverty2016, left_on='County Name', right_on='Name', how='outer') # Merge poverty with demographics
election2016 = pd.merge(election2016, mioutcomes2016, left_on='ID', right_on='county_fips', how='outer') # Merge outcomes with election2016
# now have a complete file for 2016

election2020 = pd.merge(demo2020, mipoverty2020, left_on='County Name', right_on='Name', how='outer') # Merge poverty with demographics
election2020 = pd.merge(election2020, mioutcomes2020, left_on='ID', right_on='county_fips', how='outer') # Merge outcomes with election2012
# now have a complete file for 2020

In [ ]:
# Prepare prediction data

miprediction = prediction[prediction['State Name'] == 'Nevada']

miprediction = miprediction[['County Name','Males Under 18 Percentage','Males 18-24 Percentage','Males 25-34 Percentage',
                           'Males 35-44 Percentage', 'Males 45-64 Percentage', 'Males 65+ Percentage', 'Females Under 18 Percentage',
                           'Females 18-24 Percentage', 'Females 25-34 Percentage', 'Females 35-44 Percentage', 'Females 45-64 Percentage',
                           'Females 65+ Percentage',
                           'White alone Percentage', 'Black or African American alone Percentage', 'American Indian and Alaska Native alone Percentage',
                           'Asian alone Percentage', 'Native Hawaiian and Other Pacific Islander alone Percentage',
                           'Some other race alone Percentage', 'Two or more races Percentage', 'Two or more races: Two races including Some other race Percentage',
                           'Two or more races: Two races excluding Some other race, and three or more races Percentage',
                           'No schooling completed Percentage', 'Nursery school Percentage','Kindergarten Percentage', '1st grade Percentage', '2nd grade Percentage',
                           '3rd grade Percentage', '4th grade Percentage', '5th grade Percentage', '6th grade Percentage', '7th grade Percentage', '8th grade Percentage',
                           '9th grade Percentage', '10th grade Percentage', '11th grade Percentage', '12th grade, no diploma Percentage', 'Regular high school diploma Percentage',
                           'GED or alternative credential Percentage', 'Some college, less than 1 year Percentage', 'Some college, 1 or more years, no degree Percentage',
                           "Associate's degree Percentage", "Bachelor's degree Percentage", "Master's degree Percentage", 'Professional school degree Percentage',
                           'Doctorate degree Percentage',
                           'less than $39,999 Percentage', '$40,000-$74,999 Percentage', '$75,000-$99,999 Percentage', '$100,000-$149,999 Percentage', '$150,000-$199,999 Percentage', 'more than $200,000 Percentage',]]

miprediction.head(100)
# Prediction data ready. Missing poverty statistics and median income statistics

,County Name,Males Under 18 Percentage,Males 18-24 Percentage,Males 25-34 Percentage,Males 35-44 Percentage,Males 45-64 Percentage,Males 65+ Percentage,Females Under 18 Percentage,Females 18-24 Percentage,Females 25-34 Percentage,...,Bachelor's degree Percentage,Master's degree Percentage,Professional school degree Percentage,Doctorate degree Percentage,"less than $39,999 Percentage","$40,000-$74,999 Percentage","$75,000-$99,999 Percentage","$100,000-$149,999 Percentage","$150,000-$199,999 Percentage","more than $200,000 Percentage"
257,Churchill County,0.115471,0.048408,0.072337,0.058680,0.129403,0.091739,0.590696,0.040025,0.058719,...,0.123830,0.029402,0.019868,0.003311,0.552288,0.235294,0.094118,0.060131,0.011765,0.046405
258,Clark County,0.116119,0.043159,0.074449,0.071219,0.126333,0.070759,0.609010,0.041047,0.072395,...,0.174020,0.062964,0.017342,0.009622,0.493536,0.291255,0.085932,0.074525,0.033460,0.021293
259,Douglas County,0.080787,0.031308,0.049074,0.053359,0.142574,0.149466,0.567790,0.028458,0.043132,...,0.204155,0.076050,0.021445,0.011662,0.271894,0.313618,0.132967,0.179734,0.030261,0.071527
260,Elko County,0.136549,0.047425,0.077724,0.073937,0.126511,0.061549,0.603694,0.041660,0.069925,...,0.105023,0.046283,0.013207,0.005565,0.349017,0.278398,0.136753,0.143684,0.036451,0.055696
261,Esmeralda County,0.039796,0.011224,0.032653,0.046939,0.252041,0.157143,0.518367,0.000000,0.044898,...,0.152348,0.090493,0.000000,0.000000,0.224505,0.214371,0.125553,0.175416,0.107207,0.152949
262,Eureka County,0.126387,0.038224,0.033909,0.048089,0.128853,0.128853,0.629470,0.016646,0.028360,...,0.097210,0.054905,0.000000,0.000000,0.353586,0.200844,0.071730,0.222785,0.108861,0.042194
263,Humboldt County,0.133152,0.040716,0.073439,0.063362,0.129561,0.081548,0.608885,0.038747,0.059307,...,0.154423,0.041538,0.006703,0.005468,0.092028,0.128725,0.112047,0.217593,0.168977,0.280630
264,Lander County,0.131983,0.049581,0.071404,0.081355,0.139665,0.068785,0.588163,0.014665,0.051152,...,0.066165,0.033212,0.033991,0.007006,0.180392,0.208773,0.120345,0.185770,0.130153,0.174567
265,Lincoln County,0.120036,0.042157,0.062791,0.086532,0.165298,0.081429,0.542268,0.030175,0.053694,...,0.091622,0.061814,0.000000,0.000000,0.121715,0.143473,0.117301,0.226193,0.176477,0.214841
266,Lyon County,0.108976,0.033600,0.068932,0.065601,0.135493,0.105645,0.582199,0.032590,0.059140,...,0.115436,0.034886,0.005621,0.005180,0.211024,0.240763,0.141567,0.194593,0.101704,0.110348


In [ ]:
# Put together train/test data
elections = pd.concat([election2012, election2016, election2020], axis=0, ignore_index=True)
finalelections = elections[['party', 'Males Under 18 Percentage','Males 18-24 Percentage','Males 25-34 Percentage',
                           'Males 35-44 Percentage', 'Males 45-64 Percentage', 'Males 65+ Percentage', 'Females Under 18 Percentage',
                           'Females 18-24 Percentage', 'Females 25-34 Percentage', 'Females 35-44 Percentage', 'Females 45-64 Percentage',
                           'Females 65+ Percentage',
                           'White alone Percentage', 'Black or African American alone Percentage', 'American Indian and Alaska Native alone Percentage',
                           'Asian alone Percentage', 'Native Hawaiian and Other Pacific Islander alone Percentage',
                           'Some other race alone Percentage', 'Two or more races Percentage', 'Two or more races: Two races including Some other race Percentage',
                           'Two or more races: Two races excluding Some other race, and three or more races Percentage',
                           'No schooling completed Percentage', 'Nursery school Percentage','Kindergarten Percentage', '1st grade Percentage', '2nd grade Percentage',
                           '3rd grade Percentage', '4th grade Percentage', '5th grade Percentage', '6th grade Percentage', '7th grade Percentage', '8th grade Percentage',
                           '9th grade Percentage', '10th grade Percentage', '11th grade Percentage', '12th grade, no diploma Percentage', 'Regular high school diploma Percentage',
                           'GED or alternative credential Percentage', 'Some college, less than 1 year Percentage', 'Some college, 1 or more years, no degree Percentage',
                           "Associate's degree Percentage", "Bachelor's degree Percentage", "Master's degree Percentage", 'Professional school degree Percentage',
                           'Doctorate degree Percentage',
                           'less than $39,999 Percentage', '$40,000-$74,999 Percentage', '$75,000-$99,999 Percentage', '$100,000-$149,999 Percentage', '$150,000-$199,999 Percentage', 'more than $200,000 Percentage']]

In [ ]:
# Train-test split
from sklearn.model_selection import train_test_split

# Process data:
X = finalelections[['Males Under 18 Percentage','Males 18-24 Percentage','Males 25-34 Percentage',
                           'Males 35-44 Percentage', 'Males 45-64 Percentage', 'Males 65+ Percentage', 'Females Under 18 Percentage',
                           'Females 18-24 Percentage', 'Females 25-34 Percentage', 'Females 35-44 Percentage', 'Females 45-64 Percentage',
                           'Females 65+ Percentage',
                           'White alone Percentage', 'Black or African American alone Percentage', 'American Indian and Alaska Native alone Percentage',
                           'Asian alone Percentage', 'Native Hawaiian and Other Pacific Islander alone Percentage',
                           'Some other race alone Percentage', 'Two or more races Percentage', 'Two or more races: Two races including Some other race Percentage',
                           'Two or more races: Two races excluding Some other race, and three or more races Percentage',
                           'No schooling completed Percentage', 'Nursery school Percentage','Kindergarten Percentage', '1st grade Percentage', '2nd grade Percentage',
                           '3rd grade Percentage', '4th grade Percentage', '5th grade Percentage', '6th grade Percentage', '7th grade Percentage', '8th grade Percentage',
                           '9th grade Percentage', '10th grade Percentage', '11th grade Percentage', '12th grade, no diploma Percentage', 'Regular high school diploma Percentage',
                           'GED or alternative credential Percentage', 'Some college, less than 1 year Percentage', 'Some college, 1 or more years, no degree Percentage',
                           "Associate's degree Percentage", "Bachelor's degree Percentage", "Master's degree Percentage", 'Professional school degree Percentage',
                           'Doctorate degree Percentage',
                           'less than $39,999 Percentage', '$40,000-$74,999 Percentage', '$75,000-$99,999 Percentage', '$100,000-$149,999 Percentage', '$150,000-$199,999 Percentage', 'more than $200,000 Percentage']]
y = finalelections['party']

# Train-test split:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2,random_state=100)
print(f"X_train shape: {X_train.shape}") # got it done

X_train shape: (204, 51)


In [ ]:
finalelections = finalelections.drop([34,35])


In [ ]:
total_nan_count = finalelections.isna().sum().sum()
print("Total NaNs in the DataFrame:")
print(total_nan_count)

columns_with_nan = finalelections.loc[:, finalelections.isna().any(axis=0)]
print("Columns with NaNs:")
print(columns_with_nan)


Total NaNs in the DataFrame:
1972
Columns with NaNs:
          party  Males Under 18 Percentage  Males 18-24 Percentage  \
0           NaN                        NaN                     NaN   
1           NaN                        NaN                     NaN   
2    REPUBLICAN                   0.148732                0.049457   
3    REPUBLICAN                   0.158821                0.080750   
4    REPUBLICAN                   0.267885                0.077675   
..          ...                        ...                     ...   
250    DEMOCRAT                   0.192174                0.188860   
251    DEMOCRAT                   0.250878                0.090823   
252  REPUBLICAN                   0.235537                0.076734   
253         NaN                   0.178173                0.254214   
254         NaN                   0.217091                0.090807   

     Males 25-34 Percentage  Males 35-44 Percentage  Males 45-64 Percentage  \
0                       NaN

In [ ]:
# Building the model
from sklearn.ensemble import RandomForestRegressor

# Fit model:
model = RandomForestRegressor() # Build a random forest model
rf = model.fit(X_train,y_train) # Fit the rf model

ValueError: Input contains NaN